Option chains
=======

In [1]:
from ib_insync import *
import pandas as pd

#underlying filters
min_market_cap= 2000
min_option_volume = 10000
min_iv_rank = 15

#option strategy filters
pct_under_px_range = 20
num_month_expiries = 3
max_risk = 2000
min_profit = 200
ask_tws_load =True



We want to find high ROC credit bull PUT verticals within the S&P500 constituents list, in underlyings that comply with :

* Market cap > 'min_market_cap'.
* Average option volume > 'min_option_volume'.
* Iv rank > min_iv_rank

After previous filter, scan option chains for remaining underlyings:

* Explore up to 'num_month_expiries' monthly expiries forward from current date.
* Scan combinations of strike prices starting from smallest strike to  (ATM - 'pct_under_px_range'% ) range. Zero explores all available strike combinations.
* Limit risk to 'max_risk' USD (which influences allowable distance between strikes).

Notes:
1. If notional value of a naked put strategy is less than 'max_risk' then switch to naked put strategy (do not buy long leg).
2. Order resulting strategies descending on 'Return on Capital' (MaxProfit / Margin ).
3. If 'ask_tws_load==True', user will be prompted to load strategies on TWS.
4. Only OTM options will be scaned (strikes below current price)


Get list of SP500 constituents, look for 'getSp500Constituents.py' in parent folders in order to refresh csv list file. Order descending by market cap, then sample first 'constituents_slice'

In [66]:
df = pd.read_csv( '../sp500Constituents.csv', index_col=0 )

print( df.columns )

sp500_constituents = df[['Symbol', 'Market Cap'] ]

sp500_marketCap = sp500_constituents.sort_values( by = 'Market Cap', ascending=False, axis = 0 )

sp500_marketCap = sp500_marketCap[ : constituents_slice ]

sp500_marketCap


Index(['Symbol', 'Name', 'Sector', 'Price', 'Price/Earnings', 'Dividend Yield',
       'Earnings/Share', '52 Week Low', '52 Week High', 'Market Cap', 'EBITDA',
       'Price/Sales', 'Price/Book', 'SEC Filings'],
      dtype='object')


,Symbol,Market Cap
51,AAPL,809508034020
27,GOOGL,733823966137
28,GOOG,728535558140
311,MSFT,689978437468
30,AMZN,685873374731
187,FB,523423036576
263,JPM,386613611000
261,JNJ,353062464971
185,XOM,326148660000
66,BAC,321478200969


**Connecto to IB**

In [67]:
#start ib_insync loop
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=18, readonly=True )


<IB connected to 127.0.0.1:7497 clientId=18>

**Request contracts for SP500 subset**

In [68]:
symbols = sp500_marketCap[ 'Symbol' ].values

symbols

array(['AAPL', 'GOOGL', 'GOOG', 'MSFT', 'AMZN', 'FB', 'JPM', 'JNJ', 'XOM',
       'BAC'], dtype=object)

**Create contract for each stock in the subset**

In [69]:

Contracts = [ Stock( s, 'SMART', currency='USD' ) for s in symbols ]

Contracts

[Stock(symbol='AAPL', exchange='SMART', currency='USD'),
 Stock(symbol='GOOGL', exchange='SMART', currency='USD'),
 Stock(symbol='GOOG', exchange='SMART', currency='USD'),
 Stock(symbol='MSFT', exchange='SMART', currency='USD'),
 Stock(symbol='AMZN', exchange='SMART', currency='USD'),
 Stock(symbol='FB', exchange='SMART', currency='USD'),
 Stock(symbol='JPM', exchange='SMART', currency='USD'),
 Stock(symbol='JNJ', exchange='SMART', currency='USD'),
 Stock(symbol='XOM', exchange='SMART', currency='USD'),
 Stock(symbol='BAC', exchange='SMART', currency='USD')]

**Request contract validation**

In [70]:
ib.qualifyContracts( *Contracts )

[Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'),
 Stock(conId=208813719, symbol='GOOGL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='GOOGL', tradingClass='NMS'),
 Stock(conId=208813720, symbol='GOOG', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='GOOG', tradingClass='NMS'),
 Stock(conId=272093, symbol='MSFT', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='MSFT', tradingClass='NMS'),
 Stock(conId=3691937, symbol='AMZN', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AMZN', tradingClass='NMS'),
 Stock(conId=107113386, symbol='FB', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='FB', tradingClass='NMS'),
 Stock(conId=1520593, symbol='JPM', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='JPM', tradingClass='JPM'),
 Stock(conId=8719, symbol='JNJ', exc

**Case no live market data permissions, use delayed data:**

In [15]:
#ib.reqMarketDataType(1)

**Get the tickers for each of the stock contracts. Requesting each ticker can take up to 11 seconds.**

In [71]:
tickers = []

for contract in Contracts:
    #105: avOptionVolume
    ticker = ib.reqMktData( contract, '105' )
    tickers.append(ticker)
    
print( ' {} tickers returned out of {} requested'.format( len(tickers), len(Contracts) ) )
    
#[tickers] = [  ib.reqTickers( *Contracts )]

 10 tickers returned out of 10 requested


**Apply option volume filter using ticker option volume data**

filter contracts array

In [85]:
# apply option volume filter

filteredTickers = { ticker.contract.symbol: ticker for ticker in tickers if ticker.avOptionVolume > min_option_volume }

filteredContracts = { symbol: ticker.contract for ( symbol, ticker ) in filteredTickers.items()  }

**Request market prices for remaining stocks**

Create price dictionary

In [86]:
filteredPrices = { symbol : ticker.last for ( symbol, ticker ) in filteredTickers.items()  }

**Get filtered option chains for each contract, filter by 'SMART' exchange**

In [88]:
chains = []

for ( symbol, contract ) in filteredContracts.items():
    #request al chaing
    raw_chains = ib.reqSecDefOptParams(contract.symbol, '', contract.secType, contract.conId)
    #limit to 'SMART' exchange
    chains.append( [ chain for chain in raw_chains if chain.exchange == 'SMART' ] )

print( ' {} chains found of {} requested'.format( len(chains), len(filteredContracts) ) )

 10 chains found of 10 requested


From this we can build all PUT  option contracts that meet our conditions (Bull PUTS).
  
1. Iterate over stock option 'chains', select first chain.
  1. Slice the expirations list, limit to up to 3 month expirations.
  2. Slice strike list, limit to strikes below 'pct_under_px_range'
  3. Explore remaining strike / expiriation combinations:
     1. **Expirations LOOP**. Iterate over expirations, select first expiration:
        1.  **Leg 1 LOOP**. Iterate over strikes, select leg 1 strike.
            1. Combine selected strike with current expiration from outer loop.
            2. Build Option contract.
            3. Request market data (price) for 1st leg ontract.
            4. Create combo leg 1. Fix for going to inner loop.
            5. Cancel market data (if not we would reach ticker limit), enter inner loop.
                 1. **Leg 2 LOOP**. Iterate over strikes, select leg 2 strike
                     1. Is strike < combo leg 1 strike? No : Discard / iterate again, Yes: Select it.
                     2. Combine selected strike with current expiration from outer loop
                     3. Build Option contract.
                     4. Request market data (price) for 2nd leg contract.
                     5. Cancel market data.
                     6. Create 'COMBO' order,  request 'whatif' for price / margin information.
                     7. Calculate max profit, max loss, ROC from price / margin result.
                     8. Is max profit > 'min_profit' from screener parameters? Yes: Discard.
                     9. Is max loss > 'max_risk' from screener parameters? Yes: Discard.
                     10. Is resulting ROC bigger than previous iteration? No: Discard, Yes: Save as 'l3RocWinner'.
            6. Is 'l3RocWinner' ROC better than last 'l2RocWinner'?  Yes, Save as 'l2RocWinner'
            7. Save last 'l3RocWinner' in 'L3RocWinners'.
            8. Continue to step 1 in **Leg 1 Loop**.
        2. Is 'l2RocWinner' ROC better than last 'l1RocWinner'? Yes, save as 'l1RocWinner'.
        3. Continue to step 1 in **Expirations LOOP*
  4. Results are stored in 'RocWinners' variables:
     * L3RocWinners: List of best Roc combos per short leg strike
     * L2RocWinners: List of best Roc combos per expiration
     * L1RocWinner: Best strategy overall


In [37]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P']

putContracts = [Option('SPY', expiration, strike, right, 'SMART', tradingClass='SPY')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*putContracts)
len(putContracts)

24

Select one PUT option contract as 1st leg

In [38]:

legOne = putContracts[18]

[ legOneTicker ] = ib.reqTickers( legOne )

legOne

Option(conId=395613643, symbol='SPY', lastTradeDateOrContractMonth='20200515', strike=285.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   200515P00285000', tradingClass='SPY')

In [39]:
#legOneTicker.marketPrice()

legOneTicker.last

1.33

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [40]:
print(legOneTicker.hasBidAsk())

print(legOneTicker.lastGreeks)

False
OptionComputation(impliedVol=0.27076148001280376, delta=-0.21056694838358245, optPrice=1.3300000429153442, pvDividend=0.0, gamma=0.026528949606368722, vega=0.12413070886139477, theta=-0.22764889614627715, undPrice=293.3999938964844)


build first combo leg

In [41]:
comboLegOne = ComboLeg(conId=legOne.conId, ratio= 1, action='SELL', exchange='SMART' )


Select second put contract as second leg

In [42]:
legTwo = putContracts[16]

[ legTwoTicker ] = ib.reqTickers( legTwo )

legTwo

Option(conId=395613633, symbol='SPY', lastTradeDateOrContractMonth='20200515', strike=275.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   200515P00275000', tradingClass='SPY')

In [43]:
legTwoTicker.last

0.41

In [44]:
5

False
None


Build second combo leg

In [45]:
comboLegTwo = ComboLeg(conId=legTwo.conId, ratio=1, action='BUY', exchange='SMART')

Build Contract

In [46]:
combo = Contract( symbol=spx.symbol, secType='BAG', currency='USD', exchange='SMART', comboLegs=[comboLegOne, comboLegTwo] )


#not valid since I think 'BAG' type of contracts don't really have a contract ID
#ib.qualifyContracts( combo )


Request combo ticker... So far unable to get last price or any info

In [47]:
#Seems like "combo" type contracts do not have tickers available
# (this means we might need to compute price and other parameters manually
# from leg parameters )

[comboTicker] = ib.reqTickers( combo )



In [48]:
print( comboTicker.hasBidAsk() )

comboTicker.last

#comboTicker.hasBidAsk()

False


nan

Create / Prepare (save?) combo order, should be visible in TWS

In [49]:
#! [market]

order = MarketOrder( action="BUY", totalQuantity = 1, transmit=False )


#this will have the effect of adding the order to
# tws 'Activity' pane
trade = ib.placeOrder( combo, order )
        

Trade(contract=Contract(secType='BAG', symbol='SPY', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=395613643, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=395613633, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=MarketOrder(orderId=33, clientId=15, action='BUY', totalQuantity=1, transmit=False), orderStatus=OrderStatus(orderId=33, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0, lastLiquidity=0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 5, 8, 22, 22, 6, 850990, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: . The following farms are not connected: usfarm.nj; usfuture; cashfarm; usopt; usfarm; ushmds; secdefnj.
Peer closed connection


In [29]:
ib.disconnect()